<a href="https://colab.research.google.com/github/RenjieCui/Master_Thesis/blob/main/Monte_Carlo_simulation_LQ_MFG_flocking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monte Carlo Simulation of the flocking model in the linear quadratic version

References: 

    - For intergration in Python, we use the package: 
    https://docs.scipy.org/doc/scipy/tutorial/integrate.html
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html
    
    - For Monte Carlo Simulation of Brownian motion: 
    https://quantpy.com.au/stochastic-calculus/simulating-geometric-brownian-motion-gbm-in-python/
    
    - For 3D plotting, we use the package: 
    https://gist.github.com/WetHat/1d6cd0f7309535311a539b42cccca89c

In [ ]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
from scipy.sparse import diags
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.mplot3d.proj3d import proj_transform
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime

In [ ]:
class Arrow3D(FancyArrowPatch):

    def __init__(self, x, y, z, dx, dy, dz, *args, **kwargs):
        super().__init__((0, 0), (0, 0), *args, **kwargs)
        self._xyz = (x, y, z)
        self._dxdydz = (dx, dy, dz)

    def draw(self, renderer):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        super().draw(renderer)
        
    def do_3d_projection(self, renderer=None):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))

        return np.min(zs) 

In [ ]:
def _arrow3D(ax, x, y, z, dx, dy, dz, *args, **kwargs):
    '''Add an 3d arrow to an `Axes3D` instance.'''

    arrow = Arrow3D(x, y, z, dx, dy, dz, *args, **kwargs)
    ax.add_artist(arrow)


setattr(Axes3D, 'arrow3D', _arrow3D)

# Compute for any value at time $t$

Parameters: 
1. Native to the flocking model: 
$$\kappa, \quad \sigma, \quad T, \quad d, \quad \bar{\mu}_0, \quad v_0$$
2. For simulation purpose: 
$$ M \enspace \text{Number of simulations}, \quad n \enspace \text{Number of time steps}$$

In [ ]:
#parameters in the flocking model
kappa = 1
sigma = 1
#time horizon
T = 10
#number of time steps 
n = 1000
#dimension
d = 3
#initial mean position
barmu0 = np.zeros(d)
#initial velocities 
v0 = np.zeros(d)
#number of simulation
M = 50

Functions to define:
1. eta(t): $$\eta_t = \kappa  \frac{e^{2\kappa(T-t)} -1}{e^{2\kappa(T-t)} +1}, \enspace \forall t \in [0,T]$$
2. integrate_eta(s,t): $$\text{The integral of} \enspace \eta(\cdot) \enspace \text{from time s to time t}: \int_s^t \eta_u du $$
3. chi(dim,u): $$\chi_t^i = -\kappa^2 \bar{\mu}_0^i \int_t^T e^{\int_s^t \eta_u du} \cdot ds$$
or alternatively, as defined below, with a slight modification of notations: 
$$\chi_u^i = -\kappa^2 \bar{\mu}_0^i \int_u^T e^{\int_s^u \eta_t dt} \cdot ds$$

In [ ]:
def eta(t):
    e = np.exp(1)
    return kappa * (e** (2*kappa*(T-t)) -1)/(e** (2*kappa*(T-t)) +1)

def integrate_eta(s,t):
    return integrate.quad(eta,s,t)[0]

def chi(dim, u):
    integrand = lambda s: np.exp(integrate_eta(s,u))
    dbint = integrate.quad(integrand,u,T)
    return -kappa**2 * barmu0[dim] * dbint[0]

The velocity of the representative bird can be explicitly written down as: 
$$\forall t \in (0,T], \enspace X_t = \underbrace{e^{-\int_0^t \eta_u du} v_0}_\text{part 1}  \underbrace{- \int_0^t e^{-\int_s^t \eta_u du} \chi_s ds}_\text{part 2} + \underbrace{\sigma \int_0^t e ^{-\int_s^t \eta_u du}dW_s}_\text{part 3}, \quad X_0 = v_0 \in \mathbb{R}^d $$
where $W=(W_s)_{s\in [0,T]}$ is a $d$-dimensional Brownian motion. 

Part 1: 
    $$e^{-\int_0^t \eta_u du} \cdot v_0 \in \mathbb{R}^d$$

In [ ]:
def part1(t):
    return np.exp(integrate_eta(t,0)) * v0.reshape((1,d))

Part 2: 
    $$- \int_0^t e^{-\int_s^t \eta_u du} \chi_s ds = \left[\underbrace{- \int_0^t e^{-\int_s^t \eta_u du} \chi^i_s ds}_{\in \mathbb{R}}\right]_{i=1,\cdots,d} \enspace \in \mathbb{R}^d$$
alternatively, as defined below, simply a double switch of sign and integration order, 
$$- \int_0^t e^{\int_t^s \eta_u du} \chi_s ds = \left[\underbrace{- \int_0^t e^{\int_t^s \eta_u du} \chi^i_s ds}_{\in \mathbb{R}}\right]_{i=1,\cdots,d} \enspace \in \mathbb{R}^d$$

In [ ]:
def part2_dim(dim, t):
    integrand = lambda s: np.exp(integrate_eta(t,s))* chi(dim,s) 
    dbint = integrate.quad(integrand, 0, t)
    return -dbint[0]
def part2(t):
    vec = np.zeros(d)
    for i in range(d):
        vec[i] = part2_dim(i,t)
    return vec.reshape((1,d))

Part 3: 
    $$\sigma \int_0^t e ^{-\int_s^t \eta_u du}dW_s, $$
again alternatively, with a double switch of sign and integration order, in the defined function below, we use:
$$\sigma \int_0^t e ^{\int_t^s \eta_u du}dW_s, $$
where $s$ is truncated to uniform grid of $n$ steps. 
    
One simulation: Firstly, we simulate the $d$-dimensional Brownian motion in $n$ timesteps. Then, we add the increments to the Brownian motion's path. 

We do the aforementioned simulation for $M$ times, and take average at time $t$.

Compute part 3 for a time $t \in [0,T]$:

In [ ]:
def part3_dim(t):
    dt = T/n
    increment = np.array([ np.exp(integrate_eta(t, dt*(i+1))) for i in range(n)])
    multiplier = diags(increment, 0, shape=(n,n)).toarray()
    dW = np.random.normal(0, np.sqrt(dt), size=(M,n)).T
    dS = np.matmul (multiplier, dW)  #shape (n,M)
    dS = np.vstack([np.zeros(M), dS]) #shape(n+1,M)
    S = np.cumsum(dS, axis=0) #shape(n+1, M) 
    #note: axis=0 specfices: fix a column, acculate the sum by row numbers.
    num = round(t/dt)
    return sigma * np.mean(S[num,:])
    #approximate t by the nearest grid point, 
    #and approximate the velocity-part3 at time t using the average velocity-part3 at the nearest grid point.
def part3(t):
    vec = np.zeros(d)
    for i in range(d):
        vec[i] = part3_dim(t)
    return vec.reshape((1,d))

Combine these 3 parts together for an explicit value:

In [ ]:
def velocity(t):
    return part1(t) + part2(t) + part3(t)
velocity(T)

# Simulate the terminal velocities $v_T$

Note that in the function "part3_dim(t)", we take the velocity as the average of all the simulations, so that we can have an approximate value for the velocity at time $t$.


Now, we would like to simulate the terminal velocities. And we follow the same idea from "part3_dim(t)". In other words, each Monte Carlo Simulation result is an average of $M$ simulations. On top of that, we do such Monte Carlo Simulation $M$ times. Therefore, actually, we have simulated $M \times M$ times. 

We store all of the (Monte Carlo simulated) terminal velocities in the array "terminal_velocity" which is of shape $(M,d)$. 

Recall $M$ is the number of simulations used also in the function part3_dim(t), and $d$ is the dimension. Both $d$ and $M$ are specified at the start. 

In [ ]:
def simulate_terminal_velocity():
    terminal_velocity = np.matmul(np.ones((M,1)), part1(T) + part2(T)) #shape (M,d)
    dt = T/n
    part3_terminal = np.zeros((M,d))
    for i in range(M):
        part3_terminal[i,:] = part3(T)
    terminal_velocity += part3_terminal
    return terminal_velocity

## Plot the Monte Carlo simualted terminal velocity

In [ ]:
#plot when d=2
fig = plt.figure()
data = simulate_terminal_velocity()
cmap = plt.cm.jet
cNorm  = colors.Normalize(vmin=np.min(data[:,1]), vmax=np.max(data[:,1]))
scalarMap = cmx.ScalarMappable(norm=cNorm,cmap=cmap)
for idx in range(M):
    colorVal = scalarMap.to_rgba(data[idx,1])
    dx,dy = data[idx,:]
    plt.arrow(0, 0, dx, dy, head_width=0.01, color=colorVal)
plt.xlabel("v1")
plt.ylabel("v2")
#save the file
now = datetime.now()
filename = f"barmu0={barmu0}-v0={v0}-"+now.strftime("%Y-%m-%d-%H:%M:%S")+".png"
plt.savefig(filename, dpi=300)

In [ ]:
#plot when d=3
data = simulate_terminal_velocity()
#print(data)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#x,y,z boundary
boundary = np.amax(data)   
ax.set_xlim(- boundary,boundary)
ax.set_ylim(- boundary,boundary)
ax.set_zlim(- boundary,boundary)
cmap = plt.cm.jet
cNorm  = colors.Normalize(vmin=np.min(data[:,d-1]), vmax=np.max(data[:,d-1]))
scalarMap = cmx.ScalarMappable(norm=cNorm,cmap=cmap)
for idx in range(M):
    colorVal = scalarMap.to_rgba(data[idx,d-1])
    dx,dy,dz = data[idx,:]
    ax.arrow3D(0,0,0,
           dx,dy,dz,
           mutation_scale=5,
           arrowstyle="-|>",
           color = colorVal)
ax.set_xlabel('v1')
ax.set_ylabel('v2')
ax.set_zlabel('v3')
fig.tight_layout()
#save the file
now = datetime.now()
filename = f"barmu0={barmu0}-v0={v0}-"+now.strftime("%Y-%m-%d-%H:%M:%S")+".png"
plt.savefig(filename, dpi=300)

# Simulate the whole trajectery terminated at time  $T$

### Plot for the whole trajectory terminated at time $T$ in 2D

In [ ]:
#run when d=2
#compute the determinstic velocity elements
time = np.linspace(0,T,n+1)
dt = T/n
vec_part1 = [part1(t) for t in time]
print('vec_part1')
vec_part2 = [part2(t) for t in time]
print('vec_part2')
increments = np.zeros((n+1,n))
for t in time:
    idx = round(t/dt)
    for i in range(n):
        increments[idx,i] =  np.exp(integrate_eta(t, dt*(i+1))) 
print('increments')

function "simulate_trajectory2D" produces one simulation.

assuming the initial position $x_0 = 0$

In [ ]:
#introduce stochastic factor independent in each simulation, and compute the trajectory
def simulate_trajectory2D():
    velocities = np.zeros((n+1,d))
    dW1 = np.random.normal(0, np.sqrt(dt), size=(M,n)).T
    dW2 = np.random.normal(0, np.sqrt(dt), size=(M,n)).T
    for t in time:
        idx = round(t/dt)
        multiplier = diags(increments[idx,:], 0, shape=(n,n)).toarray()
        dS1 = np.vstack([np.zeros(M), np.matmul (multiplier, dW1)])  #shape (n+1,M)
        dS2 = np.vstack([np.zeros(M), np.matmul (multiplier, dW2)])
        S1 = np.cumsum(dS1, axis=0)
        S2 = np.cumsum(dS2, axis=0)
        vec_part3 =np.array([sigma * np.mean(S1[idx,:]),  sigma * np.mean(S2[idx,:])]).reshape((1,d))
        velocities[idx,:] = vec_part1[idx]+vec_part2[idx]+ vec_part3
    trajectory = np.cumsum(dt * velocities,axis=0)
    return trajectory

In [ ]:
data = np.zeros((n+1,d,M))
for i in range(M):
    data[:,:,i] = simulate_trajectory2D()
    print(i)

In [ ]:
fig = plt.figure()
x1 = data[:,0,:] #shape(n+1,M)
x2 = data[:,1,:]
plt.plot(x1,x2)
plt.xlabel("x1")
plt.ylabel("x2")
now = datetime.now()
filename = f"2D-barmu0={barmu0}-v0={v0}-"+now.strftime("%Y-%m-%d-%H:%M:%S")+".png"
plt.savefig(filename, dpi=300)

In [ ]:
from numpy import savetxt
now = datetime.now()
filename1 = f"2Dx1-barmu0={barmu0}-v0={v0}-"+now.strftime("%Y-%m-%d-%H:%M:%S")
filename2 = f"2Dx2-barmu0={barmu0}-v0={v0}-"+now.strftime("%Y-%m-%d-%H:%M:%S")
savetxt(filename1, data[:,0,:], delimiter=',')
savetxt(filename2, data[:,1,:], delimiter=',')

### Plot for the whole trajectory terminated at time $T$ in 3D

In [ ]:
#run when d=3
#compute the determinstic velocity elements
time = np.linspace(0,T,n+1)
dt = T/n
vec_part1 = [part1(t) for t in time]
print('vec_part1')
vec_part2 = [part2(t) for t in time]
print('vec_part2')
increments = np.zeros((n+1,n))
for t in time:
    idx = round(t/dt)
    for i in range(n):
        increments[idx,i] =  np.exp(integrate_eta(t, dt*(i+1))) 
print('increments')

In [ ]:
#introduce stochastic factor independent in each simulation, and compute the trajectory
def simulate_trajectory3D():
    velocities = np.zeros((n+1,d))
    dW1 = np.random.normal(0, np.sqrt(dt), size=(M,n)).T
    dW2 = np.random.normal(0, np.sqrt(dt), size=(M,n)).T
    dW3 = np.random.normal(0, np.sqrt(dt), size=(M,n)).T
    for t in time:
        idx = round(t/dt)
        multiplier = diags(increments[idx,:], 0, shape=(n,n)).toarray()
        dS1 = np.vstack([np.zeros(M), np.matmul (multiplier, dW1)])  #shape (n+1,M)
        dS2 = np.vstack([np.zeros(M), np.matmul (multiplier, dW2)])
        dS3 = np.vstack([np.zeros(M), np.matmul (multiplier, dW3)])
        S1 = np.cumsum(dS1, axis=0)
        S2 = np.cumsum(dS2, axis=0)
        S3 = np.cumsum(dS3, axis=0)
        vec_part3 =np.array([sigma * np.mean(S1[idx,:]),  
                             sigma * np.mean(S2[idx,:]), 
                             sigma * np.mean(S3[idx,:]) ]).reshape((1,d))
        velocities[idx,:] = vec_part1[idx]+vec_part2[idx]+ vec_part3
    trajectory = np.cumsum(dt * velocities,axis=0)
    return trajectory

In [ ]:
data = np.zeros((n+1,d,M))
for i in range(M):
    data[:,:,i] = simulate_trajectory3D()
    print(i)

In [ ]:
x1 = data[:,0,:] #shape(n+1,M)
x2 = data[:,1,:]
x3 = data[:,2,:]
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for m in range(M):
    ax.plot3D(x1[:,m], x2[:,m], x3[:,m])
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_zlabel('x3')
now = datetime.now()
filename = f"3D-barmu0={barmu0}-v0={v0}-"+now.strftime("%Y-%m-%d-%H:%M:%S")+".png"
plt.savefig(filename, dpi=300)

In [ ]:
from numpy import savetxt
now = datetime.now()
filename1 = f"3Dx1-barmu0={barmu0}-v0={v0}-"+now.strftime("%Y-%m-%d-%H:%M:%S")
filename2 = f"3Dx2-barmu0={barmu0}-v0={v0}-"+now.strftime("%Y-%m-%d-%H:%M:%S")
filename3 = f"3Dx3-barmu0={barmu0}-v0={v0}-"+now.strftime("%Y-%m-%d-%H:%M:%S")
savetxt(filename1, data[:,0,:], delimiter=',')
savetxt(filename2, data[:,1,:], delimiter=',')
savetxt(filename3, data[:,1,:], delimiter=',')

# Additional codes: toy examples of plotting Brownian motions in 2D and 3D 

Toy example 1: plot a 1 dimensional Brownian motion with Monte Carlo Simulation

In [ ]:
#construct 
dt = T/n
dW =  np.random.normal(0, np.sqrt(dt), size=(M,n)).T
dW = np.vstack([np.zeros(M), dW])
W = dW.cumsum(axis =0)
#plot
time = np.linspace(0,T,n+1)
tt = np.full(shape=(M,n+1), fill_value = time).T
plt.plot(tt, W)
plt.xlabel("Time")
plt.ylabel("Value")
plt.title(f"Realizations of a 1D Brownian Motion \n $W_0 = 0, \enspace T = {T}$ \n Number of simulations: $M ={M}$")
plt.show()

Toy example 2: Plot a 3-dimensional Brownian motion

In [ ]:
T3 = 10
d3 =3
dt3 = T3 / d3
n3 =1000
dW = np.random.normal(0, np.sqrt(dt3), size=(n3,d3))
W = np.cumsum(dW, axis=0)
ax = plt.axes(projection='3d')
ax.plot3D(W[:,0], W[:,1], W[:,2], 'gray')

Toy example 3: Generate and plot a $d$-dimensional Brownian motion, d=1,2,3

In [ ]:
#generate Brownian motion
def BM1D(T,n,M): 
    dt = T/n
    dW =  np.random.normal(0, np.sqrt(dt), size=(M,n)).T
    dW = np.vstack([np.zeros(M), dW])
    W = dW.cumsum(axis =0)
    return W

def BMfull(T,n, M,d):
    W = np.zeros((d,n+1,M))
    for i in range(d):
        W[i,:,:] = BM1D(T,n,M)
    #W[i,:,:] the (i+1)-th dimension of the generated Brownian motion, having the shape (n+1,M)
    #W[:,:,m] the (m+1)-th copy of the Brownian motion, having the shape (d, n+1)
    return W

In [ ]:
#plot Brownian motion
def plotBM(T,n,M,d):
    W = BMfull(T,n, M,d)
    if d==1:
        time = np.linspace(0,T,n+1)
        tt = np.full(shape=(M,n+1), fill_value = time).T #shape(n+1,M)
        W = W[0,:,:] #shape(n+1,M)
        plt.plot(tt, W)
        plt.xlabel("Time")
        plt.ylabel("Value")
        plt.title(f"Realizations of a {d}D Brownian Motion \n $W_0 = 0, \enspace T = {T}$ \n Number of simulations: $M ={M}$")
        plt.show()
    elif d==2:
        xBM = W[0,:,:]
        yBM = W[1,:,:]
        plt.plot(xBM,yBM)
        plt.xlabel("x-axis")
        plt.ylabel("y-axis")
        plt.title(f"Realizations of a {d}D Brownian Motion \n $W_0 = 0, \enspace T = {T}$ \n Number of simulations: $M ={M}$")
        plt.show()
    elif d==3: 
        xBM = W[0,:,:]
        yBM = W[1,:,:]
        zBM = W[2,:,:]
        ax = plt.axes(projection='3d')
        for m in range(M):
            ax.plot3D(xBM[:,m], yBM[:,m], zBM[:,m])
        ax.set_xlabel('x-axis')
        ax.set_ylabel('y-axis')
        ax.set_zlabel('z-axis')
        ax.set_title(f"Realizations of a {d}D Brownian Motion \n $W_0 = 0, \enspace T = {T}$ \n Number of simulations: $M ={M}$")      